# Import and Clean Data
Eric Johnson 

## Import Libraries

In [2]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2
import scipy.stats as st
import matplotlib.pyplot as plt
import numpy as np

import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

## Import NJ/NY Bike Rental Data
Data Source: https://citibikenyc.com/system-data

In [3]:
# Read Multiple CSVs to load into a data frame

# file list will combine the path with file names for a list of file names with paths
file_list = [path + 'JC-202301-citibike-tripdata.csv', path + 'JC-202302-citibike-tripdata.csv', path + 'JC-202303-citibike-tripdata.csv', \
    path + 'JC-202304-citibike-tripdata.csv', path + 'JC-202305-citibike-tripdata.csv', path + 'JC-202306-citibike-tripdata.csv', \
    path + 'JC-202307-citibike-tripdata.csv', path + 'JC-202308-citibike-tripdata.csv', path + 'JC-202309-citibike-tripdata.csv', \
    path + 'JC-202310-citibike-tripdata.csv', path + 'JC-202311-citibike-tripdata.csv', path + 'JC-202312-citibike-tripdata.csv']

# Empty list to store data for df's
df_list = []

# loop through each file in the list, read the data, append to the list
for file in file_list:
    df = pd.read_csv(file)
    
    df_list.append(df)

# Merge into a DF    
df_merged = pd.concat(df_list)
df_merged.head(3)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0905B18B365C9D20,classic_bike,2023-01-28 09:18:10,2023-01-28 09:28:52,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Hamilton Park,JC009,40.735938,-74.030305,40.727596,-74.044247,member
1,B4F0562B05CB5404,electric_bike,2023-01-23 20:10:12,2023-01-23 20:18:27,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Southwest Park - Jackson St & Observer Hwy,HB401,40.735938,-74.030305,40.737551,-74.041664,member
2,5ABF032895F5D87E,classic_bike,2023-01-29 15:27:04,2023-01-29 15:32:38,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Marshall St & 2 St,HB408,40.735944,-74.030383,40.740802,-74.042521,member


In [5]:
# Drop Missing values
df_merged.dropna(inplace=True)
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985519 entries, 0 to 58679
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             985519 non-null  object 
 1   rideable_type       985519 non-null  object 
 2   started_at          985519 non-null  object 
 3   ended_at            985519 non-null  object 
 4   start_station_name  985519 non-null  object 
 5   start_station_id    985519 non-null  object 
 6   end_station_name    985519 non-null  object 
 7   end_station_id      985519 non-null  object 
 8   start_lat           985519 non-null  float64
 9   start_lng           985519 non-null  float64
 10  end_lat             985519 non-null  float64
 11  end_lng             985519 non-null  float64
 12  member_casual       985519 non-null  object 
dtypes: float64(4), object(9)
memory usage: 105.3+ MB


### Create a Trip Duration Measured by Minutes

In [6]:
# Convert 'started_at' and 'ended_at' columns to datetime format
# Reference to code from https://stackoverflow.com/questions/28954093/how-to-add-subtract-time-hours-minutes-etc-from-a-pandas-dataframe-index-wh
# Also referenced https://stackoverflow.com/questions/36156582/accessing-total-seconds-in-pandas-data-column

df_merged['started_at'] = pd.to_datetime(df_merged['started_at'])
df_merged['ended_at'] = pd.to_datetime(df_merged['ended_at'])

df_merged['duration_minutes'] = (df_merged['ended_at'] - df_merged['started_at']).dt.total_seconds() / 60



In [7]:
# Verify
df_merged.head(2)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration_minutes
0,0905B18B365C9D20,classic_bike,2023-01-28 09:18:10,2023-01-28 09:28:52,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Hamilton Park,JC009,40.735938,-74.030305,40.727596,-74.044247,member,10.70
1,B4F0562B05CB5404,electric_bike,2023-01-23 20:10:12,2023-01-23 20:18:27,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Southwest Park - Jackson St & Observer Hwy,HB401,40.735938,-74.030305,40.737551,-74.041664,member,8.25


### Create Distance Traveled 
Note that the distance traveled is only between start and end station and not the actual total distance traveled so any analysis with this must use caution. It also means that trips that start and end in the same station will show zero. This could be addressed by looking at the elapsed time between the trips and filling in an average but because there is so much data, I will not include the 0 values in analysis in this case. 

In [8]:
# Create a function to calculate the distance 
def haversine(lat1, lon1, lat2, lon2):

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    radius_of_earth = 6371  # Earth's radius in kilometers
    distance_km = radius_of_earth * c

    # Convert distance from kilometers to miles
    distance_miles = distance_km * 0.621371

    return distance_miles

# Calculate the distance traveled in miles
df_merged['distance_traveled_miles'] = df_merged.apply(lambda row: haversine(row['start_lat'], row['start_lng'], row['end_lat'], row['end_lng']), axis=1)

# Check the results:
df_merged.head(2)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration_minutes,distance_traveled_miles
0,0905B18B365C9D20,classic_bike,2023-01-28 09:18:10,2023-01-28 09:28:52,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Hamilton Park,JC009,40.735938,-74.030305,40.727596,-74.044247,member,10.70,0.930097
1,B4F0562B05CB5404,electric_bike,2023-01-23 20:10:12,2023-01-23 20:18:27,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Southwest Park - Jackson St & Observer Hwy,HB401,40.735938,-74.030305,40.737551,-74.041664,member,8.25,0.605049


In [9]:
# Verifying the Df info
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985519 entries, 0 to 58679
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   ride_id                  985519 non-null  object        
 1   rideable_type            985519 non-null  object        
 2   started_at               985519 non-null  datetime64[ns]
 3   ended_at                 985519 non-null  datetime64[ns]
 4   start_station_name       985519 non-null  object        
 5   start_station_id         985519 non-null  object        
 6   end_station_name         985519 non-null  object        
 7   end_station_id           985519 non-null  object        
 8   start_lat                985519 non-null  float64       
 9   start_lng                985519 non-null  float64       
 10  end_lat                  985519 non-null  float64       
 11  end_lng                  985519 non-null  float64       
 12  member_casual    

### Calculate a Speed Traveled Measured in Miles Per Hour
The speed traveled represents the speed at which the rider went from the starting station to the ending station. This makes assumptions about traveling from the stations directly. Any analysis will need to consider this limitation. Special attention should be paid to the duration of the rental compared to distance traveled and speed and compared to averages. 

In [10]:
# Calculate average ride speed in mph
df_merged['average_ride_speed_mph'] = df_merged['distance_traveled_miles'] / (df_merged['duration_minutes'] / 60)
df_merged.head(3)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration_minutes,distance_traveled_miles,average_ride_speed_mph
0,0905B18B365C9D20,classic_bike,2023-01-28 09:18:10,2023-01-28 09:28:52,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Hamilton Park,JC009,40.735938,-74.030305,40.727596,-74.044247,member,10.700000,0.930097,5.215500
1,B4F0562B05CB5404,electric_bike,2023-01-23 20:10:12,2023-01-23 20:18:27,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Southwest Park - Jackson St & Observer Hwy,HB401,40.735938,-74.030305,40.737551,-74.041664,member,8.250000,0.605049,4.400353
2,5ABF032895F5D87E,classic_bike,2023-01-29 15:27:04,2023-01-29 15:32:38,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Marshall St & 2 St,HB408,40.735944,-74.030383,40.740802,-74.042521,member,5.566667,0.718647,7.745894


### Creating Trip Length Categories

In [11]:
# Bins trip categories
bin_edges = [-0.1, 0.5, 1, 3, float('inf')]  # Define ranges for short, medium, and long trips

# Labels
bin_labels = ['Zero Miles', 'Short Trip', 'Medium Trip', 'Long Trip']

# New Column
df_merged['trip_category'] = pd.cut(df_merged['distance_traveled_miles'], bins=bin_edges, labels=bin_labels, right=False)

# Verify
df_merged.head(3)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration_minutes,distance_traveled_miles,average_ride_speed_mph,trip_category
0,0905B18B365C9D20,classic_bike,2023-01-28 09:18:10,2023-01-28 09:28:52,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Hamilton Park,JC009,40.735938,-74.030305,40.727596,-74.044247,member,10.700000,0.930097,5.215500,Short Trip
1,B4F0562B05CB5404,electric_bike,2023-01-23 20:10:12,2023-01-23 20:18:27,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Southwest Park - Jackson St & Observer Hwy,HB401,40.735938,-74.030305,40.737551,-74.041664,member,8.250000,0.605049,4.400353,Short Trip
2,5ABF032895F5D87E,classic_bike,2023-01-29 15:27:04,2023-01-29 15:32:38,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Marshall St & 2 St,HB408,40.735944,-74.030383,40.740802,-74.042521,member,5.566667,0.718647,7.745894,Short Trip


In [12]:
# Closer look
df_merged['started_at'].head()

0   2023-01-28 09:18:10
1   2023-01-23 20:10:12
2   2023-01-29 15:27:04
3   2023-01-24 18:35:08
4   2023-01-21 20:44:09
Name: started_at, dtype: datetime64[ns]

### Create Seasons
Using whole months for simplicity as opposed to the formal dates for start and end of the seasons

In [13]:
# reference stack overflow and ChatGPT for debugging https://stackoverflow.com/questions/60285557/extract-seasons-from-datetime-pandas
date = df_merged['started_at'].dt.month * 100 + df_merged['started_at'].dt.day
df_merged['season'] = (pd.cut(date, [0, 321, 620, 922, 1220, 1300],
                       labels=['winter', 'spring', 'summer', 'autumn', 'winter'], ordered=False)
                  .str.strip()
               )

# df_merged.drop('season', axis=1, inplace=True)
df_merged.sample(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration_minutes,distance_traveled_miles,average_ride_speed_mph,trip_category,season
51622,917295562D3F1BBB,classic_bike,2023-05-05 21:47:54,2023-05-05 21:54:08,City Hall - Washington St & 1 St,HB105,9 St HBLR - Jackson St & 8 St,HB305,40.737256,-74.031009,40.747907,-74.038412,member,6.233333,0.831742,8.006069,Short Trip,spring
21112,2A140EA2E0D5548B,classic_bike,2023-02-13 15:36:17,2023-02-13 15:42:01,City Hall - Washington St & 1 St,HB105,8 St & Washington St,HB603,40.737360,-74.030970,40.745984,-74.028199,casual,5.733333,0.613256,6.417790,Short Trip,winter
84390,591F76BEED33113C,classic_bike,2023-08-30 13:22:14,2023-08-30 13:29:49,Newark Ave,JC032,Newport PATH,JC066,40.721501,-74.046275,40.727224,-74.033759,member,7.583333,0.765445,6.056265,Short Trip,summer
30411,78BB9E811811FAAB,classic_bike,2023-07-07 14:43:09,2023-07-07 14:47:19,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,2 St & Park Ave,HB608,40.736982,-74.027781,40.739153,-74.033082,member,4.166667,0.315465,4.542690,Zero Miles,summer
73054,3D728867E17D2C66,classic_bike,2023-04-22 09:11:16,2023-04-22 09:32:50,Bergen Ave & Stegman St,JC108,Washington St,JC098,40.706575,-74.086701,40.724294,-74.035483,member,21.566667,2.948459,8.202822,Medium Trip,spring
47294,3CB1070AC1A9B08A,classic_bike,2023-11-14 14:41:11,2023-11-14 14:51:40,Liberty Light Rail,JC052,Exchange Pl,JC116,40.711242,-74.055701,40.716366,-74.034344,member,10.483333,1.173194,6.714626,Medium Trip,autumn
46324,E3AB64BC48A1FBA2,classic_bike,2023-06-14 18:33:27,2023-06-14 18:35:02,Bergen Ave,JC095,McGinley Square,JC055,40.722069,-74.071404,40.725340,-74.067622,member,1.583333,0.300505,11.387563,Zero Miles,spring
51949,6EBF1F4F685B5B7A,classic_bike,2023-05-19 08:56:01,2023-05-19 09:03:16,7 St & Monroe St,HB304,Hoboken Terminal - River St & Hudson Pl,HB102,40.746413,-74.037977,40.736068,-74.029127,member,7.250000,0.851783,7.049238,Short Trip,spring
35117,7ABF935365CCEFE5,classic_bike,2023-09-05 16:55:49,2023-09-05 17:00:28,Grand St & 14 St,HB506,9 St HBLR - Jackson St & 8 St,HB305,40.754090,-74.031600,40.747907,-74.038412,member,4.650000,0.556420,7.179612,Short Trip,summer
2794,474893089EB2BFD6,classic_bike,2023-01-31 21:38:29,2023-01-31 21:42:08,8 St & Washington St,HB603,City Hall - Washington St & 1 St,HB105,40.745984,-74.028199,40.737360,-74.030970,casual,3.650000,0.613256,10.080912,Short Trip,winter


### Calculating a rental cost
This will only calculate correctly for casual members who pay a per ride fee vs members who pay monthly service fees. 

In [14]:
# Reference: https://citibikenyc.com/pricing
# Calculate fee for casual members

df_merged['fee'] = np.where(df_merged['member_casual'] == 'casual',
                            np.where(df_merged['duration_minutes'] <= 30, 
                                     4.79,
                                     np.ceil(df_merged['duration_minutes'] - 30) * 0.30 + 4.79),
                            np.nan)

df_merged.sample(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,duration_minutes,distance_traveled_miles,average_ride_speed_mph,trip_category,season,fee
84268,914378E0D01A0DFC,classic_bike,2023-08-08 06:27:07,2023-08-08 06:30:15,Clinton St & 7 St,HB303,9 St HBLR - Jackson St & 8 St,HB305,40.745420,-74.033320,40.747907,-74.038412,member,3.133333,0.317118,6.072473,Zero Miles,summer,NaN
38239,318FE7F4CBC18D00,classic_bike,2023-11-22 17:58:01,2023-11-22 18:02:59,Columbus Park - Clinton St & 9 St,HB501,14 St Ferry - 14 St & Shipyard Ln,HB202,40.748230,-74.032509,40.752961,-74.024353,member,4.966667,0.537688,6.495555,Short Trip,autumn,NaN
53997,A437EC7CB8539F50,classic_bike,2023-01-05 22:37:55,2023-01-05 23:08:06,Columbus Drive,JC014,Columbus Drive,JC014,40.718355,-74.038914,40.718355,-74.038914,member,30.183333,0.000000,0.000000,Zero Miles,winter,NaN
9397,F93889CCCB2ACBE5,classic_bike,2023-07-07 12:36:42,2023-07-07 12:38:37,Warren St,JC006,Manila & 1st,JC082,40.721073,-74.038002,40.721651,-74.042884,member,1.916667,0.258760,8.100326,Zero Miles,summer,NaN
50549,3F9459AAC578301F,classic_bike,2023-03-05 16:37:33,2023-03-05 18:04:06,Grand St,JC102,Grand St,JC102,40.715178,-74.037683,40.715178,-74.037683,casual,86.550000,0.000000,0.000000,Zero Miles,winter,21.89
77625,7839AA44EE904F55,classic_bike,2023-08-11 16:38:44,2023-08-11 16:47:13,Southwest Park - Jackson St & Observer Hwy,HB401,Newport Pkwy,JC008,40.737647,-74.041673,40.728745,-74.032108,casual,8.483333,0.793169,5.609837,Short Trip,summer,4.79
39114,A4B548724703E982,classic_bike,2023-02-02 08:28:47,2023-02-02 08:38:44,Monmouth and 6th,JC075,Harborside,JC104,40.725685,-74.048790,40.719252,-74.034234,member,9.950000,0.882387,5.320927,Short Trip,winter,NaN
32002,AABD262E19B847C8,classic_bike,2023-12-28 15:28:53,2023-12-28 15:35:19,Columbus Drive,JC014,Hamilton Park,JC009,40.718808,-74.038757,40.727596,-74.044247,member,6.433333,0.671829,6.265759,Short Trip,winter,NaN
57282,44C8DE8944E2B26F,classic_bike,2023-05-13 07:50:39,2023-05-13 08:07:30,Harborside,JC104,12 St & Sinatra Dr N,HB201,40.719252,-74.034234,40.750604,-74.024020,casual,16.850000,2.231269,7.945171,Medium Trip,spring,4.79
102323,0892EC8DABD2E41C,classic_bike,2023-07-18 19:18:28,2023-07-18 19:41:54,Madison St & 1 St,HB402,Willow Ave & 12 St,HB505,40.738790,-74.039300,40.751867,-74.030377,casual,23.433333,1.017155,2.604379,Medium Trip,summer,4.79


In [15]:
# Checking closer:

funcs = ['mean', 'median', 'max', 'min', 'var', 'std', 'sem']

df_merged['fee'].agg(funcs)

mean         6.274686
median       4.790000
max       1357.490000
min          4.790000
var        159.550091
std         12.631314
sem          0.024999
Name: fee, dtype: float64

In [16]:
# Checking info again
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985519 entries, 0 to 58679
Data columns (total 19 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   ride_id                  985519 non-null  object        
 1   rideable_type            985519 non-null  object        
 2   started_at               985519 non-null  datetime64[ns]
 3   ended_at                 985519 non-null  datetime64[ns]
 4   start_station_name       985519 non-null  object        
 5   start_station_id         985519 non-null  object        
 6   end_station_name         985519 non-null  object        
 7   end_station_id           985519 non-null  object        
 8   start_lat                985519 non-null  float64       
 9   start_lng                985519 non-null  float64       
 10  end_lat                  985519 non-null  float64       
 11  end_lng                  985519 non-null  float64       
 12  member_casual    

### Trying to make data smaller 

In [17]:
# Round the 'distance_traveled_miles' column to 3 decimal places
df_merged['distance_traveled_miles'] = df_merged['distance_traveled_miles'].round(3)

# Round 'duration_minutes' column to 1 decimal place and convert to integers
df_merged['duration_minutes'] = df_merged['duration_minutes'].round(1)
df_merged['duration_minutes'] = df_merged['duration_minutes'].fillna(0).astype(int)  # Replace NaN with 0

# Round 'average_ride_speed_mph' column to 1 decimal place
df_merged['average_ride_speed_mph'] = df_merged['average_ride_speed_mph'].round(1)

# Replace non-finite values with 0
df_merged['average_ride_speed_mph'] = df_merged['average_ride_speed_mph'].replace([np.inf, -np.inf], np.nan).fillna(0)

# Convert to integers
df_merged['average_ride_speed_mph'] = df_merged['average_ride_speed_mph'].astype(int)

# Check the DataFrame info
df_merged.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 985519 entries, 0 to 58679
Data columns (total 19 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   ride_id                  985519 non-null  object        
 1   rideable_type            985519 non-null  object        
 2   started_at               985519 non-null  datetime64[ns]
 3   ended_at                 985519 non-null  datetime64[ns]
 4   start_station_name       985519 non-null  object        
 5   start_station_id         985519 non-null  object        
 6   end_station_name         985519 non-null  object        
 7   end_station_id           985519 non-null  object        
 8   start_lat                985519 non-null  float64       
 9   start_lng                985519 non-null  float64       
 10  end_lat                  985519 non-null  float64       
 11  end_lng                  985519 non-null  float64       
 12  member_casual    

### Load Individual Stations to Create Separate DF

In [18]:
# Create separate DataFrames for start and end stations
start_stations = df_merged[['start_station_id', 'start_station_name', 'start_lat', 'start_lng']]
start_stations.columns = ['station_id', 'station_name', 'lat', 'lng']

end_stations = df_merged[['end_station_id', 'end_station_name', 'end_lat', 'end_lng']]
end_stations.columns = ['station_id', 'station_name', 'lat', 'lng']

# Concatenate start and end stations
stations = pd.concat([start_stations, end_stations], ignore_index=True)

# Remove duplicate stations
stations.drop_duplicates(subset=['station_id'], inplace=True)

# Count occurrences of each station ID as both start and end stations in the original DataFrame
start_counts = df_merged['start_station_id'].value_counts().reset_index()
start_counts.columns = ['station_id', 'start_count']

end_counts = df_merged['end_station_id'].value_counts().reset_index()
end_counts.columns = ['station_id', 'end_count']

# Merge station counts with stations DataFrame
stations = pd.merge(stations, start_counts, on='station_id', how='left')
stations = pd.merge(stations, end_counts, on='station_id', how='left')

# Fill NaN values with 0
stations['start_count'].fillna(0, inplace=True)
stations['end_count'].fillna(0, inplace=True)

stations

,station_id,station_name,lat,lng,start_count,end_count
0,HB101,Hoboken Terminal - Hudson St & Hudson Pl,40.735938,-74.030305,27192.0,27310.0
1,JC009,Hamilton Park,40.727596,-74.044247,21541.0,21671.0
2,JC103,Journal Square,40.733670,-74.062500,8371.0,7879.0
3,JC102,Grand St,40.715178,-74.037683,11013.0,11003.0
4,HB402,Madison St & 1 St,40.738790,-74.039300,14400.0,14151.0
...,...,...,...,...,...,...
567,6955.05,W 54 St & 11 Ave,40.768333,-73.992573,0.0,1.0
568,6779.04,Broadway & W 51 St,40.762288,-73.983362,0.0,1.0
569,6617.02,E 53 St & 3 Ave,40.757632,-73.969306,0.0,1.0
570,7014.12,Broadway & W 61 St,40.770030,-73.981968,0.0,1.0


In [19]:
# Filter for duplicates in the station_name column
duplicate_stations = stations[stations.duplicated(subset=['station_name'], keep=False)]

duplicate_stations


,station_id,station_name,lat,lng,start_count,end_count


### Use KMeans and Clustering to Create Location Based Labels

Creating an elbow graph to determine the optimum number of K's

In [20]:
# Select columns to scale and non-scaled columns
columns_to_scale = ['lat', 'lng']
columns_to_keep = [col for col in stations.columns if col not in columns_to_scale]

# Separate data to scale and non-scaled data
data_to_scale = stations[columns_to_scale]
data_to_keep = stations[columns_to_keep]

# Scale the data
scaler = StandardScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(data_to_scale), columns=columns_to_scale, index=stations.index)

# Combine scaled and non-scaled data
scaled_stations = pd.concat([data_to_keep, scaled_data], axis=1)

scaled_stations.head()

,station_id,station_name,start_count,end_count,lat,lng
0,HB101,Hoboken Terminal - Hudson St & Hudson Pl,27192.0,27310.0,-0.120525,-0.343576
1,JC009,Hamilton Park,21541.0,21671.0,-0.396367,-0.722068
2,JC103,Journal Square,8371.0,7879.0,-0.195510,-1.217557
3,JC102,Grand St,11013.0,11003.0,-0.807017,-0.543880
4,HB402,Madison St & 1 St,14400.0,14151.0,-0.026201,-0.587767


In [21]:

# Creating Variables
inertia = []
k = list(range(1, 11))

# Looping through the K's
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1, n_init=10)
    k_model.fit(scaled_data)  
    inertia.append(k_model.inertia_)
    
# Create a Dictionary that holds the list values for k and inertia
elbow_data = {"k": k, "inertia": inertia}


# Create a DataFrame using the elbow_data Dictionary
df_elbow = pd.DataFrame(elbow_data)


# Review the DataFrame
df_elbow.head()

,k,inertia
0,1,1144.000000
1,2,619.255272
2,3,383.295650
3,4,267.423576
4,5,215.530032


In [22]:
# Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

Four Categories Will Work Based on Elbow Curve

In [23]:
# Copy the DF
stations2 = stations.copy()
stations2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 572 entries, 0 to 571
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station_id    572 non-null    object 
 1   station_name  572 non-null    object 
 2   lat           572 non-null    float64
 3   lng           572 non-null    float64
 4   start_count   572 non-null    float64
 5   end_count     572 non-null    float64
dtypes: float64(4), object(2)
memory usage: 31.3+ KB


In [24]:
# Select only 'lat' and 'lng' columns from stations2
stations_subset = stations2[['lat', 'lng']]

# Define the model with the lower value of k clusters
# Use a random_state of 1 to generate the model
model = KMeans(n_clusters=4, random_state=1, n_init=10)

# Fit the model
model.fit(stations_subset)

# Make predictions
k_4 = model.predict(stations_subset)

# Create a copy of the DataFrame and name it as spread_df_predictions
p_df = stations2.copy()

# Add a class column with the labels to the spread_df_predictions DataFrame
p_df['clusters'] = k_4

p_df.head()

,station_id,station_name,lat,lng,start_count,end_count,clusters
0,HB101,Hoboken Terminal - Hudson St & Hudson Pl,40.735938,-74.030305,27192.0,27310.0,1
1,JC009,Hamilton Park,40.727596,-74.044247,21541.0,21671.0,1
2,JC103,Journal Square,40.733670,-74.062500,8371.0,7879.0,1
3,JC102,Grand St,40.715178,-74.037683,11013.0,11003.0,1
4,HB402,Madison St & 1 St,40.738790,-74.039300,14400.0,14151.0,1


In [25]:
# Checking closer
p_df['clusters'].unique()

array([1, 0, 2, 3], dtype=int32)

In [26]:
# Changing the Cluster Values to be more human readable 
p_df['clusters'] = p_df['clusters'].replace({0: 1, 1: 2, 2: 3, 3: 4})

# Verify
p_df['clusters'].unique()

array([2, 1, 3, 4], dtype=int32)

In [27]:
# And at the info
p_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 572 entries, 0 to 571
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   station_id    572 non-null    object 
 1   station_name  572 non-null    object 
 2   lat           572 non-null    float64
 3   lng           572 non-null    float64
 4   start_count   572 non-null    float64
 5   end_count     572 non-null    float64
 6   clusters      572 non-null    int32  
dtypes: float64(4), int32(1), object(2)
memory usage: 33.5+ KB


In [28]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985519 entries, 0 to 58679
Data columns (total 19 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   ride_id                  985519 non-null  object        
 1   rideable_type            985519 non-null  object        
 2   started_at               985519 non-null  datetime64[ns]
 3   ended_at                 985519 non-null  datetime64[ns]
 4   start_station_name       985519 non-null  object        
 5   start_station_id         985519 non-null  object        
 6   end_station_name         985519 non-null  object        
 7   end_station_id           985519 non-null  object        
 8   start_lat                985519 non-null  float64       
 9   start_lng                985519 non-null  float64       
 10  end_lat                  985519 non-null  float64       
 11  end_lng                  985519 non-null  float64       
 12  member_casual    

### Adding the Cluster Groups to the Rides for both Start and Stop

In [29]:
df2 = df_merged.copy()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985519 entries, 0 to 58679
Data columns (total 19 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   ride_id                  985519 non-null  object        
 1   rideable_type            985519 non-null  object        
 2   started_at               985519 non-null  datetime64[ns]
 3   ended_at                 985519 non-null  datetime64[ns]
 4   start_station_name       985519 non-null  object        
 5   start_station_id         985519 non-null  object        
 6   end_station_name         985519 non-null  object        
 7   end_station_id           985519 non-null  object        
 8   start_lat                985519 non-null  float64       
 9   start_lng                985519 non-null  float64       
 10  end_lat                  985519 non-null  float64       
 11  end_lng                  985519 non-null  float64       
 12  member_casual    

In [30]:
df2 = df2.merge(p_df[['station_id', 'clusters']], 
                            left_on='start_station_id', 
                            right_on='station_id', 
                            how='left')

df2.rename(columns={'clusters': 'start_group'}, inplace=True)
df2.drop(columns='station_id', inplace=True)




In [31]:
df2 = df2.merge(p_df[['station_id', 'clusters']], 
                            left_on='end_station_id', 
                            right_on='station_id', 
                            how='left')

df2.rename(columns={'clusters': 'end_group'}, inplace=True)
df2.drop(columns='station_id', inplace=True)

In [32]:
df2['start_group'].nunique()

3

In [33]:
df2['end_group'].nunique()

4

In [34]:
df2['end_group'].value_counts()

2    984652
4       467
1       341
3        59
Name: end_group, dtype: int64

In [35]:
# Checking the Info
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 985519 entries, 0 to 985518
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   ride_id                  985519 non-null  object        
 1   rideable_type            985519 non-null  object        
 2   started_at               985519 non-null  datetime64[ns]
 3   ended_at                 985519 non-null  datetime64[ns]
 4   start_station_name       985519 non-null  object        
 5   start_station_id         985519 non-null  object        
 6   end_station_name         985519 non-null  object        
 7   end_station_id           985519 non-null  object        
 8   start_lat                985519 non-null  float64       
 9   start_lng                985519 non-null  float64       
 10  end_lat                  985519 non-null  float64       
 11  end_lng                  985519 non-null  float64       
 12  member_casual   

### Drop 2024 Data

In [36]:
# Convert 'ended_at' column to datetime type
df2['ended_at'] = pd.to_datetime(df2['ended_at'])

# Create a new column for the year
df2['year'] = df2['ended_at'].dt.year

# Filter rows where 'year' is 2023
df_2023 = df2[df2['year'] == 2023]

# Drop the 'year' column
df_2023.drop(columns=['year'], inplace=True)
df_2023.info()

/var/folders/t0/vnwyd4rs0cv054sd2qr9vw900000gn/T/ipykernel_73071/892220737.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023.drop(columns=['year'], inplace=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 985511 entries, 0 to 985518
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   ride_id                  985511 non-null  object        
 1   rideable_type            985511 non-null  object        
 2   started_at               985511 non-null  datetime64[ns]
 3   ended_at                 985511 non-null  datetime64[ns]
 4   start_station_name       985511 non-null  object        
 5   start_station_id         985511 non-null  object        
 6   end_station_name         985511 non-null  object        
 7   end_station_id           985511 non-null  object        
 8   start_lat                985511 non-null  float64       
 9   start_lng                985511 non-null  float64       
 10  end_lat                  985511 non-null  float64       
 11  end_lng                  985511 non-null  float64       
 12  member_casual   

Trying to connect dates correctly

In [37]:
df_2023['date_only'] = pd.to_datetime(df_2023['ended_at']).dt.date
df_2023.info()

/var/folders/t0/vnwyd4rs0cv054sd2qr9vw900000gn/T/ipykernel_73071/4210932865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2023['date_only'] = pd.to_datetime(df_2023['ended_at']).dt.date


<class 'pandas.core.frame.DataFrame'>
Int64Index: 985511 entries, 0 to 985518
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   ride_id                  985511 non-null  object        
 1   rideable_type            985511 non-null  object        
 2   started_at               985511 non-null  datetime64[ns]
 3   ended_at                 985511 non-null  datetime64[ns]
 4   start_station_name       985511 non-null  object        
 5   start_station_id         985511 non-null  object        
 6   end_station_name         985511 non-null  object        
 7   end_station_id           985511 non-null  object        
 8   start_lat                985511 non-null  float64       
 9   start_lng                985511 non-null  float64       
 10  end_lat                  985511 non-null  float64       
 11  end_lng                  985511 non-null  float64       
 12  member_casual   

In [38]:
df_2023.head(2)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,member_casual,duration_minutes,distance_traveled_miles,average_ride_speed_mph,trip_category,season,fee,start_group,end_group,date_only
0,0905B18B365C9D20,classic_bike,2023-01-28 09:18:10,2023-01-28 09:28:52,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Hamilton Park,JC009,40.735938,-74.030305,...,member,10,0.930,5,Short Trip,winter,NaN,2,2,2023-01-28
1,B4F0562B05CB5404,electric_bike,2023-01-23 20:10:12,2023-01-23 20:18:27,Hoboken Terminal - Hudson St & Hudson Pl,HB101,Southwest Park - Jackson St & Observer Hwy,HB401,40.735938,-74.030305,...,member,8,0.605,4,Short Trip,winter,NaN,2,2,2023-01-23


### Save to a New Merged CSV
This CSV will be imported into Tableau

In [39]:
# Saving the main data
# df_2023.to_csv('../data/bike.csv', index=False)


Will save the unique stations data too

In [40]:
# Saving the unique stations
# p_df.to_csv('../data/stations.csv')

## Separately import weather data
Data Source: https://www.ncei.noaa.gov/access

In [41]:
# Import separate weather file into new DF
path = '../data/raw/weather.csv'

dfw = pd.read_csv(path)

dfw.head()

/var/folders/t0/vnwyd4rs0cv054sd2qr9vw900000gn/T/ipykernel_73071/627134602.py:4: DtypeWarning: Columns (4,6,8,10,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,50,52) have mixed types. Specify dtype option on import or set low_memory=False.
  dfw = pd.read_csv(path)


,STATION,NAME,DATE,AWND,AWND_ATTRIBUTES,DAPR,DAPR_ATTRIBUTES,MDPR,MDPR_ATTRIBUTES,PGTM,...,WT04,WT04_ATTRIBUTES,WT05,WT05_ATTRIBUTES,WT06,WT06_ATTRIBUTES,WT08,WT08_ATTRIBUTES,WT11,WT11_ATTRIBUTES
0,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# Get the info
dfw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32825 entries, 0 to 32824
Data columns (total 53 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STATION          32825 non-null  object 
 1   NAME             32825 non-null  object 
 2   DATE             32825 non-null  object 
 3   AWND             2966 non-null   float64
 4   AWND_ATTRIBUTES  2966 non-null   object 
 5   DAPR             513 non-null    float64
 6   DAPR_ATTRIBUTES  513 non-null    object 
 7   MDPR             509 non-null    float64
 8   MDPR_ATTRIBUTES  509 non-null    object 
 9   PGTM             1458 non-null   float64
 10  PGTM_ATTRIBUTES  1458 non-null   object 
 11  PRCP             32185 non-null  float64
 12  PRCP_ATTRIBUTES  32185 non-null  object 
 13  SNOW             20957 non-null  float64
 14  SNOW_ATTRIBUTES  20957 non-null  object 
 15  SNWD             8204 non-null   float64
 16  SNWD_ATTRIBUTES  8204 non-null   object 
 17  TAVG        

In [43]:
dfw.shape

(32825, 53)

Too many stations. Choosing one station with a good number of readings

In [44]:
# newarkStation = dfw[dfw['STATION'] == 'USW00014734']
# newarkStation.info()

In [45]:
dfw.head()

,STATION,NAME,DATE,AWND,AWND_ATTRIBUTES,DAPR,DAPR_ATTRIBUTES,MDPR,MDPR_ATTRIBUTES,PGTM,...,WT04,WT04_ATTRIBUTES,WT05,WT05_ATTRIBUTES,WT06,WT06_ATTRIBUTES,WT08,WT08_ATTRIBUTES,WT11,WT11_ATTRIBUTES
0,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Change the D/T to match other data

In [46]:
# Adding h/m/s to the date
dfw['DATE'] = pd.to_datetime(dfw['DATE'])

dfw['DATE'] = dfw['DATE'].dt.strftime('%Y-%m-%d %H:%M:%S')

dfw['DATE'].head()

0    2023-01-01 00:00:00
1    2023-01-02 00:00:00
2    2023-01-03 00:00:00
3    2023-01-04 00:00:00
4    2023-01-05 00:00:00
Name: DATE, dtype: object

In [47]:
dfw['WT03'].head()

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: WT03, dtype: float64

### Make Data Easier to Understand
Reference to weather data: https://www.ncei.noaa.gov/pub/data/cdo/documentation/GHCND_documentation.pdf 

In [48]:
# Drop columns 
dfw = dfw.drop(['DAPR','DAPR_ATTRIBUTES','MDPR','MDPR_ATTRIBUTES','PGTM','PGTM_ATTRIBUTES', \
    'TOBS','TOBS_ATTRIBUTES','WESD','WESD_ATTRIBUTES','WESF','WESF_ATTRIBUTES'], 1)

/var/folders/t0/vnwyd4rs0cv054sd2qr9vw900000gn/T/ipykernel_73071/3324513171.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dfw = dfw.drop(['DAPR','DAPR_ATTRIBUTES','MDPR','MDPR_ATTRIBUTES','PGTM','PGTM_ATTRIBUTES', \


In [49]:
dfw.head()

,STATION,NAME,DATE,AWND,AWND_ATTRIBUTES,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,SNWD,...,WT04,WT04_ATTRIBUTES,WT05,WT05_ATTRIBUTES,WT06,WT06_ATTRIBUTES,WT08,WT08_ATTRIBUTES,WT11,WT11_ATTRIBUTES
0,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-01 00:00:00,NaN,NaN,0.27,",,N",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-02 00:00:00,NaN,NaN,0.00,",,N",0.0,",,N",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-03 00:00:00,NaN,NaN,0.11,",,N",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-04 00:00:00,NaN,NaN,0.43,",,N",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-05 00:00:00,NaN,NaN,0.03,",,N",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
dfw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32825 entries, 0 to 32824
Data columns (total 41 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STATION          32825 non-null  object 
 1   NAME             32825 non-null  object 
 2   DATE             32825 non-null  object 
 3   AWND             2966 non-null   float64
 4   AWND_ATTRIBUTES  2966 non-null   object 
 5   PRCP             32185 non-null  float64
 6   PRCP_ATTRIBUTES  32185 non-null  object 
 7   SNOW             20957 non-null  float64
 8   SNOW_ATTRIBUTES  20957 non-null  object 
 9   SNWD             8204 non-null   float64
 10  SNWD_ATTRIBUTES  8204 non-null   object 
 11  TAVG             1095 non-null   float64
 12  TAVG_ATTRIBUTES  1095 non-null   object 
 13  TMAX             5257 non-null   float64
 14  TMAX_ATTRIBUTES  5257 non-null   object 
 15  TMIN             5259 non-null   float64
 16  TMIN_ATTRIBUTES  5259 non-null   object 
 17  WDF2        

In [51]:
# Drop columns 
dfw = dfw.drop(['WT05','WT05_ATTRIBUTES','WT06','WT06_ATTRIBUTES','WT11','WT11_ATTRIBUTES'], 1)

/var/folders/t0/vnwyd4rs0cv054sd2qr9vw900000gn/T/ipykernel_73071/1785537572.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dfw = dfw.drop(['WT05','WT05_ATTRIBUTES','WT06','WT06_ATTRIBUTES','WT11','WT11_ATTRIBUTES'], 1)


In [52]:
dfw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32825 entries, 0 to 32824
Data columns (total 35 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STATION          32825 non-null  object 
 1   NAME             32825 non-null  object 
 2   DATE             32825 non-null  object 
 3   AWND             2966 non-null   float64
 4   AWND_ATTRIBUTES  2966 non-null   object 
 5   PRCP             32185 non-null  float64
 6   PRCP_ATTRIBUTES  32185 non-null  object 
 7   SNOW             20957 non-null  float64
 8   SNOW_ATTRIBUTES  20957 non-null  object 
 9   SNWD             8204 non-null   float64
 10  SNWD_ATTRIBUTES  8204 non-null   object 
 11  TAVG             1095 non-null   float64
 12  TAVG_ATTRIBUTES  1095 non-null   object 
 13  TMAX             5257 non-null   float64
 14  TMAX_ATTRIBUTES  5257 non-null   object 
 15  TMIN             5259 non-null   float64
 16  TMIN_ATTRIBUTES  5259 non-null   object 
 17  WDF2        

In [53]:
# Don't need the attributes columns either
dfw = dfw.drop(['AWND_ATTRIBUTES','PRCP_ATTRIBUTES','SNOW_ATTRIBUTES','SNWD_ATTRIBUTES','TAVG_ATTRIBUTES','TMAX_ATTRIBUTES', \
    'TMIN_ATTRIBUTES','WDF2_ATTRIBUTES','WDF5_ATTRIBUTES','WSF2_ATTRIBUTES','WSF5_ATTRIBUTES','WT01_ATTRIBUTES', \
       'WT02_ATTRIBUTES','WT03_ATTRIBUTES','WT04_ATTRIBUTES','WT08_ATTRIBUTES' ], 1)

/var/folders/t0/vnwyd4rs0cv054sd2qr9vw900000gn/T/ipykernel_73071/2023831714.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  dfw = dfw.drop(['AWND_ATTRIBUTES','PRCP_ATTRIBUTES','SNOW_ATTRIBUTES','SNWD_ATTRIBUTES','TAVG_ATTRIBUTES','TMAX_ATTRIBUTES', \


In [54]:
dfw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32825 entries, 0 to 32824
Data columns (total 19 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   STATION  32825 non-null  object 
 1   NAME     32825 non-null  object 
 2   DATE     32825 non-null  object 
 3   AWND     2966 non-null   float64
 4   PRCP     32185 non-null  float64
 5   SNOW     20957 non-null  float64
 6   SNWD     8204 non-null   float64
 7   TAVG     1095 non-null   float64
 8   TMAX     5257 non-null   float64
 9   TMIN     5259 non-null   float64
 10  WDF2     2918 non-null   float64
 11  WDF5     2916 non-null   float64
 12  WSF2     2918 non-null   float64
 13  WSF5     2916 non-null   float64
 14  WT01     663 non-null    float64
 15  WT02     59 non-null     float64
 16  WT03     199 non-null    float64
 17  WT04     27 non-null     float64
 18  WT08     211 non-null    float64
dtypes: float64(16), object(3)
memory usage: 4.8+ MB


In [55]:
# Rename columns so it's easier to read
dfw.rename(columns={'WT01':'Fog', 'WT02':'Heavy Fog', 'WT03':'Thunder', 'WT04':'Hail/Sleet', 'WT08':'Smoke/Haze'}, inplace=True)


In [56]:
dfw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32825 entries, 0 to 32824
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   STATION     32825 non-null  object 
 1   NAME        32825 non-null  object 
 2   DATE        32825 non-null  object 
 3   AWND        2966 non-null   float64
 4   PRCP        32185 non-null  float64
 5   SNOW        20957 non-null  float64
 6   SNWD        8204 non-null   float64
 7   TAVG        1095 non-null   float64
 8   TMAX        5257 non-null   float64
 9   TMIN        5259 non-null   float64
 10  WDF2        2918 non-null   float64
 11  WDF5        2916 non-null   float64
 12  WSF2        2918 non-null   float64
 13  WSF5        2916 non-null   float64
 14  Fog         663 non-null    float64
 15  Heavy Fog   59 non-null     float64
 16  Thunder     199 non-null    float64
 17  Hail/Sleet  27 non-null     float64
 18  Smoke/Haze  211 non-null    float64
dtypes: float64(16), object(3)

In [57]:
dfw['SNOW'].value_counts()

0.0    20753
0.2       30
0.1       25
0.3       24
0.5       15
0.4       12
1.0       12
1.5        9
2.5        6
0.8        6
5.0        5
0.9        5
1.2        4
2.0        4
1.8        4
3.5        3
0.7        3
1.3        2
3.0        2
5.5        2
1.4        2
0.6        2
4.8        2
2.7        2
6.0        2
4.3        2
1.6        2
1.1        2
1.9        1
2.8        1
1.7        1
4.6        1
4.4        1
5.9        1
3.1        1
5.7        1
5.1        1
3.6        1
5.3        1
2.3        1
3.2        1
4.1        1
4.5        1
Name: SNOW, dtype: int64

In [58]:
dfw['PRCP'].value_counts()

0.00    20106
0.02     1007
0.01     1004
0.03      614
0.04      467
        ...  
6.24        1
4.53        1
2.47        1
4.83        1
4.48        1
Name: PRCP, Length: 371, dtype: int64

In [59]:
dfw['Thunder'].value_counts()

1.0    199
Name: Thunder, dtype: int64

In [60]:
dfw['Fog'].value_counts()

1.0    663
Name: Fog, dtype: int64

In [61]:
# File blanks with zeros
dfw['Fog'] = dfw['Fog'].fillna(0)
dfw['Heavy Fog'] = dfw['Heavy Fog'].fillna(0)
dfw['Thunder'] = dfw['Thunder'].fillna(0)
dfw['Hail/Sleet'] = dfw['Hail/Sleet'].fillna(0)
dfw['Smoke/Haze'] = dfw['Smoke/Haze'].fillna(0)
dfw['SNOW'] = dfw['SNOW'].fillna(0.0)
dfw['SNWD'] = dfw['SNWD'].fillna(0.0)

dfw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32825 entries, 0 to 32824
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   STATION     32825 non-null  object 
 1   NAME        32825 non-null  object 
 2   DATE        32825 non-null  object 
 3   AWND        2966 non-null   float64
 4   PRCP        32185 non-null  float64
 5   SNOW        32825 non-null  float64
 6   SNWD        32825 non-null  float64
 7   TAVG        1095 non-null   float64
 8   TMAX        5257 non-null   float64
 9   TMIN        5259 non-null   float64
 10  WDF2        2918 non-null   float64
 11  WDF5        2916 non-null   float64
 12  WSF2        2918 non-null   float64
 13  WSF5        2916 non-null   float64
 14  Fog         32825 non-null  float64
 15  Heavy Fog   32825 non-null  float64
 16  Thunder     32825 non-null  float64
 17  Hail/Sleet  32825 non-null  float64
 18  Smoke/Haze  32825 non-null  float64
dtypes: float64(16), object(3)

WDF5 and WSF5 are both missing one value (missing one day's data). Will fill with an average. 

In [62]:
average = dfw['WDF5'].mean()
dfw['WDF5'] = dfw['WDF5'].fillna(average)

average = dfw['WSF5'].mean()
dfw['WSF5'] = dfw['WSF5'].fillna(average)
dfw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32825 entries, 0 to 32824
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   STATION     32825 non-null  object 
 1   NAME        32825 non-null  object 
 2   DATE        32825 non-null  object 
 3   AWND        2966 non-null   float64
 4   PRCP        32185 non-null  float64
 5   SNOW        32825 non-null  float64
 6   SNWD        32825 non-null  float64
 7   TAVG        1095 non-null   float64
 8   TMAX        5257 non-null   float64
 9   TMIN        5259 non-null   float64
 10  WDF2        2918 non-null   float64
 11  WDF5        32825 non-null  float64
 12  WSF2        2918 non-null   float64
 13  WSF5        32825 non-null  float64
 14  Fog         32825 non-null  float64
 15  Heavy Fog   32825 non-null  float64
 16  Thunder     32825 non-null  float64
 17  Hail/Sleet  32825 non-null  float64
 18  Smoke/Haze  32825 non-null  float64
dtypes: float64(16), object(3)

Testing a date without the h/m/s info

In [63]:
dfw['Date_Only'] = pd.to_datetime(dfw['DATE']).dt.date
dfw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32825 entries, 0 to 32824
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   STATION     32825 non-null  object 
 1   NAME        32825 non-null  object 
 2   DATE        32825 non-null  object 
 3   AWND        2966 non-null   float64
 4   PRCP        32185 non-null  float64
 5   SNOW        32825 non-null  float64
 6   SNWD        32825 non-null  float64
 7   TAVG        1095 non-null   float64
 8   TMAX        5257 non-null   float64
 9   TMIN        5259 non-null   float64
 10  WDF2        2918 non-null   float64
 11  WDF5        32825 non-null  float64
 12  WSF2        2918 non-null   float64
 13  WSF5        32825 non-null  float64
 14  Fog         32825 non-null  float64
 15  Heavy Fog   32825 non-null  float64
 16  Thunder     32825 non-null  float64
 17  Hail/Sleet  32825 non-null  float64
 18  Smoke/Haze  32825 non-null  float64
 19  Date_Only   32825 non-nul

In [64]:
dfw.head()

,STATION,NAME,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,Fog,Heavy Fog,Thunder,Hail/Sleet,Smoke/Haze,Date_Only
0,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-01 00:00:00,NaN,0.27,0.0,0.0,NaN,NaN,NaN,NaN,212.029835,NaN,24.327743,0.0,0.0,0.0,0.0,0.0,2023-01-01
1,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-02 00:00:00,NaN,0.00,0.0,0.0,NaN,NaN,NaN,NaN,212.029835,NaN,24.327743,0.0,0.0,0.0,0.0,0.0,2023-01-02
2,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-03 00:00:00,NaN,0.11,0.0,0.0,NaN,NaN,NaN,NaN,212.029835,NaN,24.327743,0.0,0.0,0.0,0.0,0.0,2023-01-03
3,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-04 00:00:00,NaN,0.43,0.0,0.0,NaN,NaN,NaN,NaN,212.029835,NaN,24.327743,0.0,0.0,0.0,0.0,0.0,2023-01-04
4,US1NJUN0028,"SPRINGFIELD TWP 0.7 NNE, NJ US",2023-01-05 00:00:00,NaN,0.03,0.0,0.0,NaN,NaN,NaN,NaN,212.029835,NaN,24.327743,0.0,0.0,0.0,0.0,0.0,2023-01-05


In [68]:
avgW = dfw.groupby('DATE').mean()
avgW.head()

/var/folders/t0/vnwyd4rs0cv054sd2qr9vw900000gn/T/ipykernel_73071/1748769293.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  avgW = dfw.groupby('DATE').mean()


,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,Fog,Heavy Fog,Thunder,Hail/Sleet,Smoke/Haze
DATE,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,7.058889,0.271667,0.0,0.0,51.000000,54.600000,43.266667,298.75,219.375468,19.2875,24.415137,0.078652,0.011236,0.0,0.0,0.000000
2023-01-02 00:00:00,3.952222,0.001905,0.0,0.0,49.000000,54.600000,39.533333,211.25,210.897616,10.3000,23.334544,0.023529,0.000000,0.0,0.0,0.000000
2023-01-03 00:00:00,3.853333,0.083218,0.0,0.0,49.666667,54.466667,43.533333,187.50,208.266371,11.1250,23.594896,0.043478,0.021739,0.0,0.0,0.021739
2023-01-04 00:00:00,5.393333,0.338256,0.0,0.0,55.000000,60.000000,47.066667,171.25,209.913487,12.6250,23.684312,0.068182,0.022727,0.0,0.0,0.011364
2023-01-05 00:00:00,5.343333,0.045618,0.0,0.0,49.333333,57.600000,45.866667,125.00,205.697542,12.0750,23.531898,0.076923,0.032967,0.0,0.0,0.010989


In [69]:
avgW.info()

<class 'pandas.core.frame.DataFrame'>
Index: 365 entries, 2023-01-01 00:00:00 to 2023-12-31 00:00:00
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   AWND        365 non-null    float64
 1   PRCP        365 non-null    float64
 2   SNOW        365 non-null    float64
 3   SNWD        365 non-null    float64
 4   TAVG        365 non-null    float64
 5   TMAX        365 non-null    float64
 6   TMIN        365 non-null    float64
 7   WDF2        365 non-null    float64
 8   WDF5        365 non-null    float64
 9   WSF2        365 non-null    float64
 10  WSF5        365 non-null    float64
 11  Fog         365 non-null    float64
 12  Heavy Fog   365 non-null    float64
 13  Thunder     365 non-null    float64
 14  Hail/Sleet  365 non-null    float64
 15  Smoke/Haze  365 non-null    float64
dtypes: float64(16)
memory usage: 56.6+ KB


In [71]:
avgW['PRCP'].value_counts()

0.000000    86
0.008652     2
0.000110     2
0.000115     2
0.013605     2
            ..
1.842473     1
0.062842     1
0.059892     1
0.065161     1
0.000769     1
Name: PRCP, Length: 271, dtype: int64

In [72]:
avgW['SNOW'].value_counts()

0.000000    347
1.985882      1
0.048421      1
0.003659      1
0.121978      1
0.002299      1
0.326966      1
0.001087      1
0.050000      1
0.085366      1
0.005495      1
0.040244      1
0.001111      1
0.003488      1
0.090000      1
0.011957      1
0.002410      1
0.046316      1
0.164835      1
Name: SNOW, dtype: int64

In [100]:
# Merge the weather data with the rideshare data using a left join
merged_df = pd.merge(avgW, df_2023, how='left', left_index=True, right_on='date_only')

# Group by date and count the number of rentals
weather_df = merged_df.groupby('date_only').size().reset_index(name='rental_count')

# Print the first few rows of weather_df
weather_df.head()


,date_only,rental_count
0,2023-01-01,1659
1,2023-01-02,1732
2,2023-01-03,1447
3,2023-01-04,2044
4,2023-01-05,2082


In [105]:
avgW_reset_index = avgW.reset_index()
print(avgW_reset_index.columns)

concatenated_avgW = pd.concat([avgW_reset_index, weather_df], axis=1)

avgW['Daily Bike Rentals'] = concatenated_avgW['rental_count']

avgW.head()

Index(['DATE', 'AWND', 'PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN', 'WDF2',
       'WDF5', 'WSF2', 'WSF5', 'Fog', 'Heavy Fog', 'Thunder', 'Hail/Sleet',
       'Smoke/Haze', 'Daily Bike Rentals'],
      dtype='object')


,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,Fog,Heavy Fog,Thunder,Hail/Sleet,Smoke/Haze,Daily Bike Rentals
DATE,,,,,,,,,,,,,,,,,
2023-01-01,7.058889,0.271667,0.0,0.0,51.000000,54.600000,43.266667,298.75,219.375468,19.2875,24.415137,0.078652,0.011236,0.0,0.0,0.000000,NaN
2023-01-02,3.952222,0.001905,0.0,0.0,49.000000,54.600000,39.533333,211.25,210.897616,10.3000,23.334544,0.023529,0.000000,0.0,0.0,0.000000,NaN
2023-01-03,3.853333,0.083218,0.0,0.0,49.666667,54.466667,43.533333,187.50,208.266371,11.1250,23.594896,0.043478,0.021739,0.0,0.0,0.021739,NaN
2023-01-04,5.393333,0.338256,0.0,0.0,55.000000,60.000000,47.066667,171.25,209.913487,12.6250,23.684312,0.068182,0.022727,0.0,0.0,0.011364,NaN
2023-01-05,5.343333,0.045618,0.0,0.0,49.333333,57.600000,45.866667,125.00,205.697542,12.0750,23.531898,0.076923,0.032967,0.0,0.0,0.010989,NaN


In [106]:
avgW['Daily Bike Rentals'].value_counts()

Series([], Name: Daily Bike Rentals, dtype: int64)

In [120]:
# Merge avgW and weather_df DataFrames using left merge on the index
merged_avgW = pd.merge(avgW, weather_df, how='left', left_index=True, right_index=True)

# Assign 'rental_count' column to 'Daily Bike Rentals' in avgW DataFrame
avgW['Daily Bike Rentals'] = merged_avgW['rental_count']

# Display the head of avgW DataFrame to verify the changes
avgW.head()


,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,Fog,Heavy Fog,Thunder,Hail/Sleet,Smoke/Haze,Daily Bike Rentals
DATE,,,,,,,,,,,,,,,,,
2023-01-01,7.058889,0.271667,0.0,0.0,51.000000,54.600000,43.266667,298.75,219.375468,19.2875,24.415137,0.078652,0.011236,0.0,0.0,0.000000,1659
2023-01-02,3.952222,0.001905,0.0,0.0,49.000000,54.600000,39.533333,211.25,210.897616,10.3000,23.334544,0.023529,0.000000,0.0,0.0,0.000000,1732
2023-01-03,3.853333,0.083218,0.0,0.0,49.666667,54.466667,43.533333,187.50,208.266371,11.1250,23.594896,0.043478,0.021739,0.0,0.0,0.021739,1447
2023-01-04,5.393333,0.338256,0.0,0.0,55.000000,60.000000,47.066667,171.25,209.913487,12.6250,23.684312,0.068182,0.022727,0.0,0.0,0.011364,2044
2023-01-05,5.343333,0.045618,0.0,0.0,49.333333,57.600000,45.866667,125.00,205.697542,12.0750,23.531898,0.076923,0.032967,0.0,0.0,0.010989,2082


In [122]:
avgW['Daily Bike Rentals'].value_counts()

2082    3
3708    3
2151    2
3360    2
1930    2
       ..
3508    1
2997    1
2566    1
2651    1
1445    1
Name: Daily Bike Rentals, Length: 342, dtype: int64

In [121]:
avgW.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 365 entries, 2023-01-01 to 2023-12-31
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AWND                365 non-null    float64
 1   PRCP                365 non-null    float64
 2   SNOW                365 non-null    float64
 3   SNWD                365 non-null    float64
 4   TAVG                365 non-null    float64
 5   TMAX                365 non-null    float64
 6   TMIN                365 non-null    float64
 7   WDF2                365 non-null    float64
 8   WDF5                365 non-null    float64
 9   WSF2                365 non-null    float64
 10  WSF5                365 non-null    float64
 11  Fog                 365 non-null    float64
 12  Heavy Fog           365 non-null    float64
 13  Thunder             365 non-null    float64
 14  Hail/Sleet          365 non-null    float64
 15  Smoke/Haze          365 non-null    fl

In [73]:
avgW.to_csv('../data/weather_cleaned.csv')